In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install sklearn
# !pip install matplotlib

In [1]:
#This is dev branch
import datetime
import math
import copy
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import logging
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

NUM_TIME_SLOTS = 144
NUM_DAYS_IN_DATA = 0

In [ ]:
# 24 hours is divided into 144 slots where each slot is 10 mins long
def calculateTimeSlot(time,printValue=True):
    global NUM_TIME_SLOTS
    dateTime = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    timePart = dateTime.time()
    timeInMinutes = (timePart.hour * 60) + timePart.minute + (timePart.second/60) + 1
    timeSlot = timeInMinutes/10
    roundedTimeSlot = math.ceil(timeSlot)
    if roundedTimeSlot > NUM_TIME_SLOTS:
        roundedTimeSlot -= 1
    if printValue==True:
        print(f"time: {time} timeInMinutes: {timeInMinutes} timeSlot: {roundedTimeSlot}")
    return int(roundedTimeSlot)

def extractDayOfWeek(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return d.weekday()

def extractDate(time):
    d = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return d.date()

print(type(extractDate('2019-01-01 00:00:00')))

In [ ]:
def extractNumberOfPOI(poiArray):
    poiCount = 0
    for poiEntry in poiArray:
        poiEntry = poiEntry + ':'
        divisions = poiEntry.split(":")
        poiCount += int(divisions[1])
    return poiCount
        

In [ ]:
def readMultipleData(path,fileNamePrefix,headerNames,dataTypes):
    global NUM_DAYS_IN_DATA
    filesToExplore = []
    for file in os.listdir(path):
        if file.startswith(fileNamePrefix):
            filesToExplore.append(file)
            # print(f"{file} read")
        else:
            continue

    print(f"{len(filesToExplore)} files read")
    if fileNamePrefix == 'order':
        NUM_DAYS_IN_DATA = len(filesToExplore)
    
    readData = []
    for files in filesToExplore:
        fileRead = pd.read_csv(path + files, sep='\t', names=headerNames,dtype=dataTypes)
        readData.append(fileRead)

    readData = pd.concat(readData, ignore_index=True)
    return readData

In [ ]:
# now region Data
regionData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
# print(regionData.head())
regionData.to_csv('regionData.csv',index=False)

In [ ]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderDataPath = './training_data/order_data/'
orderData = readMultipleData(orderDataPath,'order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'], dataTypes)
print("printing order data")
# print(orderData.head())


In [ ]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherDataPath = './training_data/weather_data/'
weatherData = readMultipleData(weatherDataPath,'weather', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
# print(weatherData.head())

In [ ]:
weatherData['time_slot'] = weatherData['time'].apply(calculateTimeSlot,printValue=False)
weatherData['date'] = weatherData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherData = weatherData.drop(['temperature','PM2.5','time'], axis=1)
print(weatherData.head())

In [ ]:
orderData = pd.merge(regionData,orderData, how='left', right_on='start_region_hash', left_on='region_hash')
orderData['time_slot'] = orderData['time'].apply(calculateTimeSlot,printValue=False)
orderData['day_of_week'] = orderData['time'].apply(extractDayOfWeek)
orderData['date'] = orderData['time'].apply(extractDate)
orderData = orderData.drop(['passenger_id', 'dest_region_hash','start_region_hash','price'], axis=1)
print(orderData.head())
# regionData=None

In [ ]:
mergedOrderData = pd.merge(weatherData,orderData, how="inner", on=['date','time_slot'])
print(mergedOrderData.head())
# orderData.to_csv('orderData.csv',index=False)

In [ ]:
orderData=None
orderData = mergedOrderData
# orderData.to_csv('mergedOrderData.csv',index=False)
# type(orderData['driver_id'][4])==float

In [ ]:
orderData = pd.read_csv('mergedOrderData.csv')

In [ ]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./training_data/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiData['poi_count'] = poiData['poi_class'].apply(extractNumberOfPOI)
poiData = pd.merge(regionData,poiData, how='inner', on='region_hash')
poiData = poiData.drop(['region_hash'], axis=1)
poiData = poiData.drop(['poi_class'], axis=1)
print(poiData.head())

In [ ]:
orderData = pd.merge(orderData,poiData, how="inner", on='region_id')
print(orderData.head())

In [ ]:
# print(mergedDataCSV)
orderData['requests'] = 1
# print(orderData.head())
groupedMergedData = orderData.groupby(['region_id','time_slot','day_of_week','weather','poi_count'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['region_id','time_slot','day_of_week','weather','poi_count'])
print(orderData.head())

In [ ]:
orderData = orderData.drop(["requests_x"], axis=1)
orderData = orderData.rename(columns={"requests_y": "requests"})
print(orderData.head())

In [ ]:
orderData['temp'] = 1
orderData['answers'] = orderData['temp'].where(orderData['driver_id'].notnull(), 0)
orderData = orderData.drop(['temp'], axis=1)
groupedMergedData = orderData.groupby(['region_id','time_slot','day_of_week','weather','poi_count'])['answers'].agg('sum').reset_index()
orderData = pd.merge(orderData,groupedMergedData, how='left' , on=['region_id','time_slot','day_of_week','weather','poi_count'])
print(orderData)

In [ ]:
orderData = orderData.drop(["answers_x"], axis=1)
orderData = orderData.rename(columns={"answers_y": "answers"})
orderData.to_csv('mergedOrderData.csv',index=False)

In [2]:
orderData = pd.read_csv('mergedOrderData.csv')

In [3]:
orderData['supply_demand'] = orderData['requests'] - orderData['answers']
print(orderData)

          weather  time_slot        date                       region_hash   
0               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a  \
1               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
2               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
3               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
4               1          1  2016-01-01  90c5a34f06ac86aee0fd70e2adce7d8a   
...           ...        ...         ...                               ...   
12222999        2        141  2016-01-21  a735449c5c09df639c35a7d61fad3ee5   
12223000        2        142  2016-01-21  a735449c5c09df639c35a7d61fad3ee5   
12223001        2        142  2016-01-21  a735449c5c09df639c35a7d61fad3ee5   
12223002        2        142  2016-01-21  a735449c5c09df639c35a7d61fad3ee5   
12223003        2        142  2016-01-21  a735449c5c09df639c35a7d61fad3ee5   

          region_id                          order_id   
0     

In [14]:
# Apply model here
AImodel = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
X = orderData[['region_id','time_slot','day_of_week','weather','poi_count']]
Y = orderData['supply_demand']
AImodel.fit(X,Y)

RandomForestRegressor(max_depth=5, random_state=0)

In [24]:
tempData = pd.DataFrame([[1,5,3,7,20000]],columns=['region_id','time_slot','day_of_week','weather','poi_count'])
# tempData = pd.DataFrame([[1,5,3,7,20000]],columns=['region_id','time_slot'])
print(AImodel.predict(tempData))
mse = mean_squared_error([28], AImodel.predict(tempData))
print(mse)

[28.65673643]
0.4313027409787012


In [29]:
# make rolling window function
def rollTheWindow(data,windowSize):
    datasetLength = len(data)
    listOfPredictions = []
    numberOfWindows = datasetLength//windowSize
    for i in range(numberOfWindows):
        rollingWindow = []
        lowerLimit = i
        upperLimit = i+windowSize
        rollingWindow.append(data[lowerLimit:upperLimit])
        rollingWindow = pd.concat(rollingWindow)
        X_WINDOW_INPUT = rollingWindow[['region_id','time_slot','day_of_week','weather','poi_count']]
        Y_WINDOW_OUTPUT = rollingWindow['supply_demand']
        currentPrediction = AImodel.predict(X_WINDOW_INPUT)
        meanSqError = mean_squared_error(Y_WINDOW_OUTPUT, currentPrediction)
        print(f"Prediction for window [{lowerLimit},{upperLimit}]={currentPrediction} MSE={meanSqError}")
        predictionTuple = (lowerLimit,upperLimit,currentPrediction,meanSqError)
        listOfPredictions.append(predictionTuple)
        i = upperLimit + 1
    return listOfPredictions

In [31]:
rollTheWindow(orderData,1)

Prediction for window [0,1]=[124.21251258] MSE=11709.947878351923
Prediction for window [1,2]=[124.21251258] MSE=11709.947878351923
Prediction for window [2,3]=[124.21251258] MSE=11709.947878351923
Prediction for window [3,4]=[124.21251258] MSE=11709.947878351923
Prediction for window [4,5]=[124.21251258] MSE=11709.947878351923
Prediction for window [5,6]=[124.21251258] MSE=11709.947878351923
Prediction for window [6,7]=[124.21251258] MSE=11709.947878351923
Prediction for window [7,8]=[124.21251258] MSE=11709.947878351923
Prediction for window [8,9]=[124.21251258] MSE=11709.947878351923
Prediction for window [9,10]=[124.21251258] MSE=11709.947878351923
Prediction for window [10,11]=[124.21251258] MSE=11709.947878351923
Prediction for window [11,12]=[124.21251258] MSE=11709.947878351923
Prediction for window [12,13]=[124.21251258] MSE=11709.947878351923
Prediction for window [13,14]=[124.21251258] MSE=11709.947878351923
Prediction for window [14,15]=[124.21251258] MSE=11709.947878351923

KeyboardInterrupt: 

In [ ]:
# read test data
# read region Data
regionTestData = pd.read_csv('./test_set/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
print(regionTestData.head())
#regionData.to_csv('regionData.csv',index=False)

In [ ]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str',  'time':'str'}
orderDataPath = './test_set/order_data/'
orderTestData = readMultipleData(orderDataPath,'test_order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'time'], dataTypes,delimiter2=",")
print("printing order data")
print(orderTestData.head())

In [ ]:
# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherTestDataPath = './test_set/weather_data/'
weatherTestData = readMultipleData(weatherTestDataPath,'weather_data', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
print(weatherTestData.head())

In [ ]:
#pre processing weather Test Data based on weather timeSlot 
weatherTestData['time_slot'] = weatherTestData['time'].apply(calculateTimeSlot,printValue=False)
weatherTestData['date'] = weatherTestData['time'].apply(extractDate)
# weatherData['day_of_week'] = weatherData['time'].apply(extractDayOfWeek)
weatherTestData = weatherTestData.drop(['temperature','PM2.5','time'], axis=1)
print(weatherTestData.head())

In [ ]:
orderTestData = pd.merge(regionTestData,orderTestData, how='inner', right_on='start_region_hash', left_on='region_hash')
orderTestData['time_slot'] = orderTestData['time'].apply(calculateTimeSlot,printValue=False)
orderTestData['day_of_week'] = orderTestData['time'].apply(extractDayOfWeek)
orderTestData['date'] = orderTestData['time'].apply(extractDate)
orderTestData = orderTestData.drop(['passenger_id','start_region_hash'], axis=1)
print(orderTestData.head())
# regionData=None

In [ ]:
mergedTestOrderData = pd.merge(weatherTestData,orderTestData, how="inner", on=['date','time_slot'])
print(mergedTestOrderData.head())
# orderData.to_csv('orderData.csv',index=False)

In [ ]:
orderTestData=None
orderTestData = mergedTestOrderData

In [ ]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./test_set/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiTestData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
poiTestData['poi_count'] = poiTestData['poi_class'].apply(extractNumberOfPOI)
poiTestData = pd.merge(regionTestData,poiTestData, how='inner', on='region_hash')
poiTestData = poiTestData.drop(['region_hash'], axis=1)
poiTestData = poiTestData.drop(['poi_class'], axis=1)
print(poiTestData.head())

In [ ]:
orderTestData = pd.merge(orderTestData,poiTestData, how="inner", on='region_id')
orderTestData = orderTestData.drop(["poi_count_x"],axis=1)
orderTestData = orderTestData.rename(columns={"poi_count_y": "poi_count"})
print(orderTestData.head())

In [ ]:
# print(mergedDataCSV)
orderTestData['requests'] = 1
# print(orderData.head())
groupedTestMergedData = orderTestData.groupby(['region_id','time_slot','day_of_week','weather','poi_count'])['requests'].agg('sum').reset_index()
# groupedMergedData = groupedMergedData.drop(['date','region_hash','order_id','driver_id','time'], axis=1)
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['region_id','time_slot','day_of_week','weather','poi_count'])
print(orderTestData.head())

In [ ]:
orderTestData = orderTestData.drop(["requests_x"], axis=1)
orderTestData = orderTestData.rename(columns={"requests_y": "requests"})
print(orderTestData.head())

In [ ]:
orderTestData['temp'] = 1
orderTestData['answers'] = orderTestData['temp'].where(orderTestData['driver_id'].notnull(), 0)
orderTestData = orderTestData.drop(['temp'], axis=1)
groupedTestMergedData = orderTestData.groupby(['region_id','time_slot','day_of_week','weather','poi_count'])['answers'].agg('sum').reset_index()
orderTestData = pd.merge(orderTestData,groupedTestMergedData, how='left' , on=['region_id','time_slot','day_of_week','weather','poi_count'])
print(orderTestData)

In [ ]:
orderTestData = orderTestData.drop(["answers_x"], axis=1)
orderTestData = orderTestData.rename(columns={"answers_y": "answers"})
orderTestData.to_csv('mergedOrderData.csv',index=False)

In [ ]:
orderTestData['supply_demand'] = orderTestData['requests'] - orderTestData['answers']
print(orderTestData)